In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
# trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
# devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
# testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


12152
2867
3005


In [4]:
print(testSentences[3])

[['Die', 'O'], ['sechs', 'O'], ['Abgeordneten', 'O'], ['der', 'O'], ['Grünen', 'B-ORG'], [',', 'O'], ['die', 'O'], ['seit', 'O'], ['Bildung', 'O'], ['einer', 'O'], ['Großen', 'O'], ['Koalition', 'O'], ['von', 'O'], ['Christ-', 'O'], ['und', 'O'], ['Sozialdemokraten', 'O'], ['die', 'O'], ['Opposition', 'O'], ['darstellen', 'O'], [',', 'O'], ['hatten', 'O'], ['sich', 'O'], ['beim', 'O'], ['Vorsteher', 'O'], ['Rainer', 'B-PER'], ['Bergert', 'I-PER'], ['für', 'O'], ['ihr', 'O'], ['Fehlen', 'O'], ['entschuldigen', 'O'], ['lassen', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0
n_gt_100 = 0
for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)
        if len(sentence) > 100:
            n_gt_100 += 1
print(n_gt_100)

0


In [6]:
print(len(labelSet))
print(models.max_sequence_length)

9
100


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-LOC': 1, 'I-PER': 5, 'B-MISC': 2, 'B-LOC': 8, 'B-ORG': 6, 'PADDING_TOKEN': 0, 'I-MISC': 7, 'O': 3, 'B-PER': 4, 'I-ORG': 9}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'allUpper': 3, 'mainly_numeric': 6, 'other': 5, 'PADDING_TOKEN': 0, 'contains_digit': 7, 'initialUpper': 4, 'numeric': 1, 'allLower': 2}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'Ü': 52, 'F': 40, 'C': 53, '§': 54, 'E': 85, 'g': 6, "'": 22, 'w': 7, 'M': 56, ',': 57, '*': 9, '?': 43, 'T': 10, 'Z': 59, 'Y': 62, 'G': 11, 'z': 12, '+': 63, '<W>': 3, 'U': 13, 'p': 14, '6': 15, 'e': 16, '"': 17, 'I': 55, 'Ä': 64, '1': 46, 'i': 18, 'b': 65, ';': 19, 'O': 66, '2': 67, 'ô': 28, '9': 20, 'í': 21, 'K': 68, '-': 23, '4': 8, 'S': 25, 'u': 26, 'è': 27, 'A': 70, '&': 29, 'ê': 90, '(': 73, '8': 72, 'ó': 31, 'ñ': 74, 'ò': 75, 'Ö': 60, 'k': 71, 'UNKNOWN': 103, 'c': 76, 'ì': 30, 'P': 32, 'á': 77, 'B': 78, 'j': 34, '%': 80, '7': 24, 'ä': 101, '</S>': 2, 'y': 79, 'h': 35, 'PADDING_TOKEN': 0, 'ç': 81, 'q': 82, 'd': 83, '<S>': 1, 'é': 84, '.': 58, ')': 69, '/': 61, 'n': 86, '0': 87, 'o': 36, 'H': 88, 'Q': 89, 'v': 37, 'l': 91, 'R': 100, 'ü': 92, 'ß': 38, 's': 39, 'J': 41, '=': 42, 't': 93, '!': 94, ':': 95, '</W>': 4, 'x': 96, 'V': 33, 'ö': 97, 'D': 44, 'f': 98, 'm': 45, '3': 99, 'L': 5, 'W': 47, 'N': 50, 'X': 102, '5': 48, 'a': 49, 'r': 51}


In [14]:
print(trainSentences[0]) 

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
5


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

10
10


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm_conll.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm_v2()

In [ ]:
#import importlib
#importlib.reload(utils)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm_conll.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm_conll.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_lstm_v2()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 3328        char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('model_lstm_conll.9.h5', '../models/final_model_conll.h5')
shutil.copyfile('model_lstm_conll.9.h5.indexes', '../models/final_model_conll.indexes')

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_conll.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('conll_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = " ".join([tok[0], correctlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close